# OpenAPI Toolkit

We can construct agents to consume arbitrary APIs, here APIs conformant to the `OpenAPI`/`Swagger` specification.

In [32]:
# NOTE: In this example. We must set `allow_dangerous_request=True` to enable the OpenAPI Agent to automatically use the Request Tool.
# This can be dangerous for calling unwanted requests. Please make sure your custom OpenAPI spec (yaml) is safe.
ALLOW_DANGEROUS_REQUEST = True

## 1st example: hierarchical planning agent

In this example, we'll consider an approach called hierarchical planning, common in robotics and appearing in recent works for LLMs X robotics. We'll see it's a viable approach to start working with a massive API spec AND to assist with user queries that require multiple steps against the API.

The idea is simple: to get coherent agent behavior over long sequences behavior & to save on tokens, we'll separate concerns: a "planner" will be responsible for what endpoints to call and a "controller" will be responsible for how to call them.

In the initial implementation, the planner is an LLM chain that has the name and a short description for each endpoint in context. The controller is an LLM agent that is instantiated with documentation for only the endpoints for a particular plan. There's a lot left to get this working very robustly :)

---

### To start, let's collect some OpenAPI specs.

In [15]:
import os

import yaml

You will be able to get OpenAPI specs from here: [APIs-guru/openapi-directory](https://github.com/APIs-guru/openapi-directory)

In [16]:
# Run these steps only if we do not have Swagger yaml files of these APIs
#!wget https://raw.githubusercontent.com/openai/openai-openapi/master/openapi.yaml -O openai_openapi.yaml
#!wget https://raw.githubusercontent.com/APIs-guru/openapi-directory/main/APIs/spotify.com/1.0.0/openapi.yaml -O spotify_openapi.yaml

In [17]:
from langchain_community.agent_toolkits.openapi.spec import reduce_openapi_spec

In [18]:
with open("openai_openapi.yaml") as f:
    raw_openai_api_spec = yaml.load(f, Loader=yaml.Loader)
openai_api_spec = reduce_openapi_spec(raw_openai_api_spec)

with open("spotify_openapi.yaml") as f:
    raw_spotify_api_spec = yaml.load(f, Loader=yaml.Loader)
spotify_api_spec = reduce_openapi_spec(raw_spotify_api_spec)

---

We'll work with the Spotify API as one of the examples of a somewhat complex API. There's a bit of auth-related setup to do if you want to replicate this.

- You'll have to set up an application in the Spotify developer console, documented [here](https://developer.spotify.com/documentation/general/guides/authorization/), to get credentials: `CLIENT_ID`, `CLIENT_SECRET`, and `REDIRECT_URI`.
- To get an access tokens (and keep them fresh), you can implement the oauth flows, or you can use `spotipy`. If you've set your Spotify creedentials as environment variables `SPOTIPY_CLIENT_ID`, `SPOTIPY_CLIENT_SECRET`, and `SPOTIPY_REDIRECT_URI`, you can use the helper functions below:

In [19]:
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
os.environ['SPOTIPY_CLIENT_ID']=os.getenv("SPOTIPY_CLIENT_ID")
os.environ['SPOTIPY_CLIENT_SECRET']=os.getenv("SPOTIPY_CLIENT_SECRET")
os.environ['SPOTIPY_REDIRECT_URI']=os.getenv("SPOTIPY_REDIRECT_URI")


In [21]:
import spotipy.util as util
from langchain_community.utilities.requests import RequestsWrapper


def construct_spotify_auth_headers(raw_spec: dict):
    scopes = list(
        raw_spec["components"]["securitySchemes"]["oauth_2_0"]["flows"][
            "authorizationCode"
        ]["scopes"].keys()
    )
    access_token = util.prompt_for_user_token(scope=",".join(scopes))
    return {"Authorization": f"Bearer {access_token}"}


# Get API credentials.
headers = construct_spotify_auth_headers(raw_spotify_api_spec)
requests_wrapper = RequestsWrapper(headers=headers)

# Notes: Make sure to enter the redirected link, such as: https://thangtrandev.net/?code=AQAyQl1aPpQs99k4MMqXeZBAzINpugn8Acb-E12BsZaNym0ztFJbe6ozNWCapD4Zcfv3NbDVo0_hoDnThKWH7yZl4qzU3gJ52jKA6RVmByqhDQyNa8iA35SY6r_AazJ1G24l0Yn2Y5lsp3wHmhTiygcs65TRMh_a9jyzZ0twu74uZB93Q4xkcVQe_QLFKQOuRChJw4G_FblvCy8wPqockd3FYqnAOo0uPApNwul6sJN8urRGH4nYgDl72t0E_QVSvD_vr5VadZ65Ejbb8jGS0gnLzzeEvJnRyL8lzlyCtGY3roJq1nUMp94xs9n3LJjdnW9mw_kBACVUmUpmvEI5Oh1RNtZOnd05BATNKfDo8ITsOQ2pC_9oDeQyQZpSSf9e4Y-r4-6TVCQ7HjIv1CrgZlvrv-NFLhXLsaHAEu_Q0YxQGyqpJgdjwGlJTet9lamNK4XCwAT3dWC1Trl_tiA8uliATOvusBUAp5gkkilg6WnMwXGTz035qevBGEY0WgkBWwQHa4MZGELjibI4gu5VgKXyiJJV6RpQCefNM2W5HdERfzsw-Mzx293Dn3AK2hndZyHlWB0nDgTPAxoQU1OQ2HLFDS7M0rg_tfIKw-Zj-iLePgKa552i54T-h_G0wTc2MNvA0q6MmvdlJBLl4XyJSbb2XcFV0gg3Fdih4PyABUKPwYXw6t2Can9UBXy3OVl7PY1v2AyyQ-EdCURprfM

### How big is this spec?

In [22]:
endpoints = [
    (route, operation)
    for route, operations in raw_spotify_api_spec["paths"].items()
    for operation in operations
    if operation in ["get", "post"]
]
len(endpoints)

63

In [23]:
import tiktoken

enc = tiktoken.encoding_for_model("gpt-4")


def count_tokens(s):
    return len(enc.encode(s))


count_tokens(yaml.dump(raw_spotify_api_spec))

68910

### Let's see some examples!

Starting with GPT-4. (Some robustness iterations under way for GPT-3 family.)

In [33]:
from langchain_community.agent_toolkits.openapi import planner
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.0)

In [34]:
# NOTE: set allow_dangerous_requests manually for security concern https://python.langchain.com/docs/security
spotify_agent = planner.create_openapi_agent(
    spotify_api_spec,
    requests_wrapper,
    llm,
    allow_dangerous_requests=ALLOW_DANGEROUS_REQUEST,
)
user_query = (
    "make me a playlist with the first song from kind of blue. call it machine blues."
)
spotify_agent.invoke(user_query)



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to find the right API calls to create a playlist with the first song from the album "Kind of Blue" and name it "Machine Blues"
Observation: 1. GET /albums with a query param to search for "Kind of Blue" to get the album's Spotify ID.
2. GET /albums/{id}/tracks to get the tracks from the album "Kind of Blue". Take the Spotify ID of the first track.
3. GET /me to get the current user's Spotify ID.
4. POST /users/{user_id}/playlists to create a new playlist named "Machine Blues".
5. POST /playlists/{playlist_id}/tracks to add the first song from the album "Kind of Blue" to the newly created playlist.
Thought:I'm ready to execute the API calls.
Action: api_controller
Action Input: 1. GET /albums with a query param to search for "Kind of Blue" to get the album's Spotify ID.
2. GET /albums/{id}/tracks to get the tracks from the album "Kind of Blue". Take the Spotify ID of the first track.
3. GET /me to get the c

RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for gpt-4 in organization org-CKrg0v0PHxuOodrFd2y0RurI on tokens per min (TPM): Limit 10000, Requested 26403. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [35]:
user_query = "give me a song I'd like, make it blues-ey"
spotify_agent.invoke(user_query)



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to find the right API calls to get a blues song recommendation for the user
Observation: 1. GET /me to get the current user's profile information.
2. GET /recommendations/available-genre-seeds to get a list of available genres.
3. GET /recommendations with a query param to get a blues song recommendation for the user.
Thought:I'm ready to execute the API calls.
Action: api_controller
Action Input: 1. GET /me to get the current user's profile information.
2. GET /recommendations/available-genre-seeds to get a list of available genres.
3. GET /recommendations with a query param to get a blues song recommendation for the user.

> Entering new AgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for gpt-4 in organization org-CKrg0v0PHxuOodrFd2y0RurI on tokens per min (TPM): Limit 10000, Requested 16804. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

#### Try another API.


In [36]:
headers = {"Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}"}
openai_requests_wrapper = RequestsWrapper(headers=headers)

In [37]:
# Meta!
llm = ChatOpenAI(model_name="gpt-4", temperature=0.25)
openai_agent = planner.create_openapi_agent(
    openai_api_spec, openai_requests_wrapper, llm
)
user_query = "generate a short piece of advice"
openai_agent.invoke(user_query)



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: generate a short piece of advice
Observation: Sorry, this API's domain is data management and machine learning, not generating advice.
Thought:The API doesn't support generating advice. I should inform the user.
Final Answer: I'm sorry, but this API's domain is data management and machine learning, not generating advice.

> Finished chain.


{'input': 'generate a short piece of advice',
 'output': "I'm sorry, but this API's domain is data management and machine learning, not generating advice."}

Takes awhile to get there!

## 2nd example: "json explorer" agent

Here's an agent that's not particularly practical, but neat! The agent has access to 2 toolkits. One comprises tools to interact with json: one tool to list the keys of a json object and another tool to get the value for a given key. The other toolkit comprises `requests` wrappers to send GET and POST requests. This agent consumes a lot calls to the language model, but does a surprisingly decent job.


In [30]:
from langchain_community.agent_toolkits import OpenAPIToolkit, create_openapi_agent
from langchain_community.tools.json.tool import JsonSpec
from langchain_openai import OpenAI

In [31]:
with open("openai_openapi.yaml") as f:
    data = yaml.load(f, Loader=yaml.FullLoader)
json_spec = JsonSpec(dict_=data, max_value_length=4000)


openapi_toolkit = OpenAPIToolkit.from_llm(
    OpenAI(temperature=0), json_spec, openai_requests_wrapper, verbose=True
)
openapi_agent_executor = create_openapi_agent(
    llm=OpenAI(temperature=0),
    toolkit=openapi_toolkit,
    allow_dangerous_requests=ALLOW_DANGEROUS_REQUEST,
    verbose=True,
)

ValueError: You must set allow_dangerous_requests to True to use this tool. Requests can be dangerous and can lead to security vulnerabilities. For example, users can ask a server to make a request to an internal server. It's recommended to use requests through a proxy server and avoid accepting inputs from untrusted sources without proper sandboxing.Please see: https://python.langchain.com/docs/security for further security information.

In [ ]:
openapi_agent_executor.run(
    "Make a post request to openai /completions. The prompt should be 'tell me a joke.'"
)



> Entering new AgentExecutor chain...
Action: json_explorer
Action Input: What is the base url for the API?

> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['openapi', 'info', 'servers', 'tags', 'paths', 'components', 'x-oaiMeta']
Thought: I should look at the servers key to see what the base url is
Action: json_spec_list_keys
Action Input: data["servers"][0]
Observation: ValueError('Value at path `data["servers"][0]` is not a dict, get the value directly.')
Thought: I should get the value of the servers key
Action: json_spec_get_value
Action Input: data["servers"][0]
Observation: {'url': 'https://api.openai.com/v1'}
Thought: I now know the base url for the API
Final Answer: The base url for the API is https://api.openai.com/v1

> Finished chain.

Observation: The base url for the API is https://api.openai.com/v1
Thought: I should find the path for the /completions endpoint.
Action: json_explorer
Action Input: What is the path for th

'The response of the POST request is {"id":"cmpl-70Ivzip3dazrIXU8DSVJGzFJj2rdv","object":"text_completion","created":1680307139,"model":"davinci","choices":[{"text":" with mummy not there”\\n\\nYou dig deep and come up with,","index":0,"logprobs":null,"finish_reason":"length"}],"usage":{"prompt_tokens":4,"completion_tokens":16,"total_tokens":20}}'